In [34]:
!pip install AugmentedSocialScientist pandas numpy wordcloud matplotlib spacy nltk
!python -m spacy download fr_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 20.2 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')


REMARQUE : BESOIN D'AVOIR FAIT TOURNE DESCRIPTION ANNOTATIONS POUR CREER LE FICHIER AVEC LES PHRASES !

In [1]:
import numpy as np
import random
import torch
import sklearn
random_seed = 1968
# Set random seed for NumPy
np.random.seed(random_seed)

# Set random seed for Python built-in random module
random.seed(random_seed)

# Set random seed for PyTorch
torch.manual_seed(random_seed)

# Set random seed for Scikit-learn
sklearn.utils.check_random_state(random_seed)


RandomState(MT19937) at 0x1DB3E554D40

In [5]:
import pandas as pd

current_annotations_file = "../../data/intermediate/annotations/personne_withSentence_1400annotations_2024-03-27_ActiveTigger.csv"

annotations = pd.read_csv(current_annotations_file, dtype = {'sentence_id':'int', 
'Personne':'str','sentence':'str'}).set_index('sentence_id')

annotations.head(5)

,Personne,sentence
sentence_id,,
1220587,Corps humain,Il importe donc de tenir compte de toutes les ...
1360306,Personne responsable et autonome,Ces personnes donnent un consentement qui relè...
580205,Personne responsable et autonome,"Sur le plan éthique, cette évolution peut être..."
700014,Personne responsable et autonome,C'est dans ce cadre que peut se\nposer la ques...
870085,Personne à protéger,"Si le sujet est majeur, \nson refus de transfu..."


In [6]:
len(annotations)

1400

# Bert classification binaire

In [43]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

from torch import cuda

cuda.get_device_name(0)

'Tesla T4'

In [44]:
from sklearn.model_selection import train_test_split
 
train_data, test_data = train_test_split(annotations, train_size=0.80, random_state=random_seed, stratify=annotations["Personne"])

In [45]:
from AugmentedSocialScientist.models import Camembert

bert = Camembert()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


There are 1 GPU(s) available.
We will use GPU 0: Tesla T4


In [46]:
train_data['text'] = train_data['sentence']
test_data['text'] = test_data['sentence']

for cat in ["Corps humain","Genre/espèce humaine","Individu ou sujet",
            "Personne à protéger","Personne relationnelle","Personne responsable et autonome"]:
    train_data['label'] = (train_data['Personne']==cat)
    test_data['label'] = (test_data['Personne']==cat)

    batch_size = 16
    train_loader = bert.encode(train_data.text.values,
    train_data.label.values, batch_size=batch_size)

    test_loader = bert.encode(test_data.text.values,
    test_data.label.values, batch_size=batch_size)

    scores = bert.run_training(
        train_loader,
        test_loader,
        n_epochs=3,
        lr=2e-5,
        random_state=random_seed,
        save_model_as=cat
        )

    categories = ["Corps humain","Genre/espèce humaine","Individu ou sujet",
        "Personne"]
    pd.DataFrame({
        "category" : categories,
        "precision" : scores[0],
        "recall" : scores[1],
        "f1-score" : scores[2],
        "support" : scores[3]
    }).set_index("category").to_csv(f"./models/{cat}/{cat}_scores.csv")

    pd.DataFrame({
        "category" : categories,
        "precision" : scores[0],
        "recall" : scores[1],
        "f1-score" : scores[2],
        "support" : scores[3]
    }).set_index("category")

  0%|          | 0/732 [00:00<?, ?it/s]

  0%|          | 0/732 [00:00<?, ?it/s]

label ids: {'Corps humain': 0, 'Genre/espèce humaine': 1, 'Individu ou sujet': 2, 'Personne': 3}
